## Import

In [10]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
print(os.environ.get('CUDA_VISIBLE_DEVICES'))

2


In [11]:
import os
import cv2
from PIL import Image
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


# CUDA 장치 사용 가능 여부 확인
if torch.cuda.is_available():
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True

cuda


## Utils

In [12]:
# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

## Custom Dataset

In [13]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        directory_path = "/mnt/nas27/Dataset/Samsung_DM"
        img_path = self.data.iloc[idx, 1]
        img_path = os.path.join(directory_path, img_path[2:])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image
        
        mask_path = self.data.iloc[idx, 2]
        mask_path = os.path.join(directory_path, mask_path[2:])
        mask = cv2.imread(mask_path)
        #mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        mask[mask == 255] = 12 #배경을 픽셀값 12로 간주

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

## Data Loader

In [14]:
transform = A.Compose(
    [   
        #A.Resize(224, 224),
        A.Resize(256, 256),
        A.Normalize(),
        ToTensorV2()
    ]
)

dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./train_source.csv'), transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)
valid_dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./val_source.csv'), transform=transform)
valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=4)

## Define Model

In [15]:
# U-Net의 기본 구성 요소인 Double Convolution Block을 정의합니다.
def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True)
    )

# 간단한 U-Net 모델 정의
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.dconv_down1 = double_conv(3, 64)
        self.dconv_down2 = double_conv(64, 128)
        self.dconv_down3 = double_conv(128, 256)
        self.dconv_down4 = double_conv(256, 512)

        self.maxpool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.dconv_up3 = double_conv(256 + 512, 256)
        self.dconv_up2 = double_conv(128 + 256, 128)
        self.dconv_up1 = double_conv(128 + 64, 64)

        self.conv_last = nn.Conv2d(64, 13, 1) # 12개 class + 1 background

    def forward(self, x):
        conv1 = self.dconv_down1(x)
        x = self.maxpool(conv1)

        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)

        conv3 = self.dconv_down3(x)
        x = self.maxpool(conv3)   

        x = self.dconv_down4(x)

        x = self.upsample(x)        
        x = torch.cat([x, conv3], dim=1)

        x = self.dconv_up3(x)
        x = self.upsample(x)        
        x = torch.cat([x, conv2], dim=1)

        x = self.dconv_up2(x)
        x = self.upsample(x)        
        x = torch.cat([x, conv1], dim=1)

        x = self.dconv_up1(x)

        out = self.conv_last(x)

        return out

## Model Train

In [16]:
# # model 초기화
# model = UNet().to(device)

# # loss function과 optimizer 정의
# criterion = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# # training loop
# for epoch in tqdm(range(20)):  # 20 에폭 동안 학습합니다.
#     model.train()
#     epoch_loss = 0
#     for images, masks in tqdm(dataloader):
#         images = images.float().to(device)
#         masks = masks.long().to(device)

#         optimizer.zero_grad()
#         outputs = model(images)
#         loss = criterion(outputs, masks.squeeze(1))
#         loss.backward()
#         optimizer.step()

#         epoch_loss += loss.item()

#     print(f'Epoch {epoch+1}, Loss: {epoch_loss/len(dataloader)}')

# Validation

In [17]:
# valid_dataset = CustomDataset(csv_file='./val_source.csv', transform=transform, infer=True)
# valid_dataloader = DataLoader(valid_dataset, batch_size=16, shuffle=False, num_workers=4)

# # 클래스(레이블) 수
# num_classes = 12

# # 클래스별 IoU를 계산하기 위한 함수
# def calculate_iou_per_class(y_true, y_pred, class_id):
#     intersection = np.sum((y_true == class_id) & (y_pred == class_id))
#     union = np.sum((y_true == class_id) | (y_pred == class_id))
#     iou = intersection / union if union > 0 else 0
#     return iou

# with torch.no_grad():
#     model.eval()
#     result = []
#     valid_true = []
#     for images, masks in tqdm(valid_dataloader):
#         images = images.float().to(device)
#         outputs = model(images)
#         outputs = torch.softmax(outputs, dim=1).cpu()
#         outputs = torch.argmax(outputs, dim=1).numpy()
#         # 클래스별 IoU를 계산하고 기록
#         class_ious = []
#         for class_id in range(num_classes):
#             iou = calculate_iou_per_class(masks, np.array(outputs), class_id)
#             class_ious.append(iou)
#         # batch에 존재하는 각 이미지에 대해서 반복
#         for pred in outputs:
#             pred = pred.astype(np.uint8)
#             pred = Image.fromarray(pred) # 이미지로 변환
#             pred = pred.resize((960, 540), Image.NEAREST) # 960 x 540 사이즈로 변환
#             pred = np.array(pred) # 다시 수치로 변환
#             # class 0 ~ 11에 해당하는 경우에 마스크 형성 / 12(배경)는 제외하고 진행
#             for class_id in range(12):
#                 class_mask = (pred == class_id).astype(np.uint8)
#                 if np.sum(class_mask) > 0: # 마스크가 존재하는 경우 encode
#                     mask_rle = rle_encode(class_mask)
#                     result.append(mask_rle)
#                 else: # 마스크가 존재하지 않는 경우 -1
#                     result.append(-1)
#         # Validation 데이터셋의 정답 이미지를 로드하여 mIoU 계산


# # 클래스별 IoU 출력
# for class_id, iou in enumerate(class_ious):
#     print(f'Class {class_id} IoU: {iou:.4f}')

# # mIoU 계산
# mIoU = np.mean(class_ious)
# print(f"mIoU Score: {mIoU:.4f}")

In [18]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="Design_PJ",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.005,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 15,
    }
)
# model 초기화
model = UNet().to(device)

# loss function과 optimizer 정의
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 클래스(레이블) 수
num_classes = 12

# 클래스별 IoU를 계산하기 위한 함수
def calculate_iou_per_class(y_true, y_pred, class_id):
    intersection = np.sum((y_true == class_id) & (y_pred == class_id))
    union = np.sum((y_true == class_id) | (y_pred == class_id))
    iou = intersection / union if union > 0 else 0
    return iou


# training loop
for epoch in range(30):  # 5 에폭 동안 학습합니다.
          
    # 클래스별 IoU를 누적할 리스트 초기화
    class_ious = []
    train_class_ious = []
    # 학습
    model.train()
    epoch_loss = 0
    
    for images, masks in tqdm(dataloader):
        images = images.float().to(device)
        masks = masks.long().to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks.squeeze(1))
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        # validation 클래스별 IoU 계산
        outputs = torch.softmax(outputs, dim=1).cpu()
        outputs = torch.argmax(outputs, dim=1).numpy()

        for class_id in range(num_classes):
            iou = calculate_iou_per_class(np.array(masks.cpu()), np.array(outputs), class_id)
            class_ious.append(iou)
            train_class_ious.append(iou)

    # mIoU 계산
    train_mIoU = np.mean(class_ious)
    train_class_ious = np.array(train_class_ious).reshape(num_classes, -1)
    train_class_ious = np.mean(train_class_ious, axis=1)
    #mIoU = np.mean(class_ious)
    #print(f"Epoch {epoch+1}, Train Loss: {epoch_loss/len(dataloader)}, train mIoU Score: {mIoU:.4f}")
    for class_id, iou in enumerate(train_class_ious):
        print(f'Class {class_id} IoU: {iou:.4f}')

    # validation
    val_loss = 0
    class_ious = []  # 클래스별 IoU를 누적할 리스트 초기화
    val_class_ious = []
    with torch.no_grad():
        model.eval()
        for images, masks in tqdm(valid_dataloader):
            images = images.float().to(device)
            masks = masks.long().to(device)
            outputs = model(images)
            # validation loss 계산
            val_loss += criterion(outputs, masks.squeeze(1)).item()

            # validation 클래스별 IoU 계산
            outputs = torch.softmax(outputs, dim=1).cpu()
            outputs = torch.argmax(outputs, dim=1).numpy()

            for class_id in range(num_classes):
                iou = calculate_iou_per_class(np.array(masks.cpu()), np.array(outputs), class_id)
                class_ious.append(iou)
                val_class_ious.append(iou)

    val_class_ious = np.array(val_class_ious).reshape(num_classes, -1)
    val_class_ious = np.mean(val_class_ious, axis=1)
    for class_id, iou in enumerate(val_class_ious):
        print(f'Class {class_id} IoU: {iou:.4f}')

                
    # for class_id, iou in enumerate(class_ious):
    # print(f'Class {class_id} IoU: {iou:.4f}')           
   
    # mIoU 계산
    val_mIoU = np.mean(class_ious)

    # log metrics to wandb
    wandb.log({"Loss1": (epoch_loss/len(dataloader))})
    wandb.log({"Train Score": train_mIoU})
    wandb.log({"Loss2": (val_loss/len(valid_dataloader))})
    wandb.log({"Validation Score": val_mIoU})
    
# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Loss1,█▄▃▂▂▁▁
Loss2,█▆▂▂▃▃▁
Train Score,▁▃▄▅▆▇█
Validation Score,▁▂▆▆▇▇█
Loss1,0.35027
Loss2,0.63945
Train Score,0.44631
Validation Score,0.31592


100%|██████████| 69/69 [01:28<00:00,  1.28s/it]


Class 0 IoU: 0.0186
Class 1 IoU: 0.0192
Class 2 IoU: 0.0198
Class 3 IoU: 0.0617
Class 4 IoU: 0.0760
Class 5 IoU: 0.0955
Class 6 IoU: 0.0918
Class 7 IoU: 0.1207
Class 8 IoU: 0.1556
Class 9 IoU: 0.1081
Class 10 IoU: 0.1416
Class 11 IoU: 0.1541


100%|██████████| 15/15 [00:18<00:00,  1.23s/it]


...

Class 0 IoU: 0.7384
Class 1 IoU: 0.7065
Class 2 IoU: 0.7145
Class 3 IoU: 0.7082
Class 4 IoU: 0.7231
Class 5 IoU: 0.6991
Class 6 IoU: 0.7225
Class 7 IoU: 0.7079
Class 8 IoU: 0.7419
Class 9 IoU: 0.6989
Class 10 IoU: 0.7262
Class 11 IoU: 0.7078


100%|██████████| 15/15 [00:18<00:00,  1.25s/it]


Class 0 IoU: 0.4079
Class 1 IoU: 0.3453
Class 2 IoU: 0.4572
Class 3 IoU: 0.3493
Class 4 IoU: 0.4065
Class 5 IoU: 0.3475
Class 6 IoU: 0.4441
Class 7 IoU: 0.3151
Class 8 IoU: 0.4384
Class 9 IoU: 0.3413
Class 10 IoU: 0.4413
Class 11 IoU: 0.3579


 88%|████████▊ | 61/69 [01:20<00:10,  1.32s/it]


KeyboardInterrupt: 

## Inference

In [ ]:
# #test_dataset = CustomDataset(csv_file='./test.csv', transform=transform, infer=True)
# test_dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./val_source.csv'), transform=transform, infer=True)
# test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

In [ ]:

# # 클래스(레이블) 수
# num_classes = 12

# # training loop
# for epoch in range(1):  # 5 에폭 동안 학습합니다.
          
#     # 클래스별 IoU를 누적할 리스트 초기화
#     class_ious = []
    
#     # validation
#     val_loss = 0
#     class_ious = []  # 클래스별 IoU를 누적할 리스트 초기화
#     with torch.no_grad():
#         model.eval()
#         for images, masks in tqdm(valid_dataloader):
#             images = images.float().to(device)
#             masks = masks.long().to(device)
#             outputs = model(images)

#             # validation loss 계산
#             val_loss += criterion(outputs, masks.squeeze(1)).item()

#             # validation 클래스별 IoU 계산
#             outputs = torch.softmax(outputs, dim=1).cpu()
#             outputs = torch.argmax(outputs, dim=1).numpy()

#             for class_id in range(num_classes):
#                 iou = calculate_iou_per_class(np.array(masks.cpu()), np.array(outputs), class_id)
#                 class_ious.append(iou)
                
#     for class_id, iou in enumerate(class_ious):
#         print(f'Class {class_id} IoU: {iou:.4f}')           
   
#     # mIoU 계산
#     val_mIoU = np.mean(class_ious)



100%|██████████| 30/30 [00:17<00:00,  1.76it/s]

Class 0 IoU: 0.5627
Class 1 IoU: 0.2968
Class 2 IoU: 0.3439
Class 3 IoU: 0.0222
Class 4 IoU: 0.0591
Class 5 IoU: 0.1001
Class 6 IoU: 0.1081
Class 7 IoU: 0.5658
Class 8 IoU: 0.5288
Class 9 IoU: 0.0046
Class 10 IoU: 0.0000
Class 11 IoU: 0.4843
Class 12 IoU: 0.5184
Class 13 IoU: 0.1730
Class 14 IoU: 0.3405
Class 15 IoU: 0.0980
Class 16 IoU: 0.0629
Class 17 IoU: 0.1000
Class 18 IoU: 0.3161
Class 19 IoU: 0.5353
Class 20 IoU: 0.6153
Class 21 IoU: 0.0000
Class 22 IoU: 0.0000
Class 23 IoU: 0.5852
Class 24 IoU: 0.5033
Class 25 IoU: 0.0919
Class 26 IoU: 0.2842
Class 27 IoU: 0.2456
Class 28 IoU: 0.0666
Class 29 IoU: 0.2117
Class 30 IoU: 0.2798
Class 31 IoU: 0.5545
Class 32 IoU: 0.4554
Class 33 IoU: 0.0000
Class 34 IoU: 0.0000
Class 35 IoU: 0.5953
Class 36 IoU: 0.5745
Class 37 IoU: 0.1012
Class 38 IoU: 0.1893
Class 39 IoU: 0.1849
Class 40 IoU: 0.0381
Class 41 IoU: 0.1408
Class 42 IoU: 0.1441
Class 43 IoU: 0.5200
Class 44 IoU: 0.4901
Class 45 IoU: 0.0000
Class 46 IoU: 0.0000
Class 47 IoU: 0.5690
Cl

In [ ]:
# with torch.no_grad():
#     model.eval()
#     result = []
#     for images in tqdm(test_dataloader):
#         images = images.float().to(device)
#         outputs = model(images)
#         outputs = torch.softmax(outputs, dim=1).cpu()
#         outputs = torch.argmax(outputs, dim=1).numpy()
#         # batch에 존재하는 각 이미지에 대해서 반복
#         for pred in outputs:
#             pred = pred.astype(np.uint8)
#             pred = Image.fromarray(pred) # 이미지로 변환
#             pred = pred.resize((960, 540), Image.NEAREST) # 960 x 540 사이즈로 변환
#             pred = np.array(pred) # 다시 수치로 변환
#             # class 0 ~ 11에 해당하는 경우에 마스크 형성 / 12(배경)는 제외하고 진행
#             for class_id in range(12):
#                 class_mask = (pred == class_id).astype(np.uint8)
#                 if np.sum(class_mask) > 0: # 마스크가 존재하는 경우 encode
#                     mask_rle = rle_encode(class_mask)
#                     result.append(mask_rle)
#                 else: # 마스크가 존재하지 않는 경우 -1
#                     result.append(-1)
    


  0%|          | 0/30 [00:03<?, ?it/s]


ValueError: operands could not be broadcast together with shapes (2,224,224) (16,224,224) 

## Submission

In [ ]:
# submit = pd.read_csv('./sample_submission.csv')
# submit['mask_rle'] = result
# submit

,id,mask_rle
0,TEST_0000_class_0,477314 4 478274 4 479242 5 479255 9 480202 5 4...
1,TEST_0000_class_1,-1
2,TEST_0000_class_2,1 167 601 527 1561 531 2521 531 3481 531 4441 ...
3,TEST_0000_class_3,462824 13 463784 13 464744 13 465704 17 466664...
4,TEST_0000_class_4,-1
...,...,...
22771,TEST_1897_class_7,944 17 1904 17 2860 4 3820 4 4780 4 5757 4 671...
22772,TEST_1897_class_8,112 528 682 120 1072 528 1642 120 2037 518 260...
22773,TEST_1897_class_9,209002 9 209962 9 220021 4 220981 4 221937 4 2...
22774,TEST_1897_class_10,-1


In [ ]:
# submit.to_csv('C:\Workspace\Design_project\submission/baseline_submit.csv', index=False)